In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

# Load the data
train_df = pd.read_csv("./split-data/train_new.csv")
test_df = pd.read_csv("./split-data/test_new.csv")
val_df = pd.read_csv("./split-data/val_new.csv")

In [ ]:
# decrease the validation set size so there are at most 100 images in each category
val_df = val_df.groupby('category').head(100)

In [ ]:
# Create the image data generators
traingen=ImageDataGenerator(rescale= 1./255)
train_generator=traingen.flow_from_dataframe(
    dataframe=train_df,
    directory="./dataset",
    x_col="img_path",
    y_col= "category",
    batch_size=64,
    shuffle= True,
    target_size = (256,256),
    class_mode="categorical",
    workers=4
)

# Create the validation data generator
valgen = ImageDataGenerator(rescale=1./255)
val_generator = valgen.flow_from_dataframe(
    dataframe=val_df,
    directory="./dataset",
    x_col="img_path",
    y_col="category",
    target_size=(256, 256),
    batch_size=64,
    class_mode="categorical",
    workers=4
)

Found 15889 validated image filenames belonging to 46 classes.
Found 3139 validated image filenames belonging to 46 classes.


In [ ]:
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.layers import Dense
from tensorflow.keras.applications.densenet import DenseNet121

In [ ]:
# Load pretrained DenseNet121 model
# Make classifer deeper
# Add new features (dropout, etc)
# Make more layers trainable (last 1-2)
base_model = DenseNet121(weights='imagenet', include_top=False, pooling='avg', input_shape=(256, 256, 3))
for i in range(0, len(base_model.layers) - 2):
    base_model.layers[i].trainable = False

# Create new model
num_classes = 46
layer1 = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(base_model.output)
dropout1 = Dropout(0.5)(layer1)
layer2 = Dense(512, activation='relu', kernel_regularizer=l2(0.001))(dropout1)
dropout2 = Dropout(0.5)(layer2)
output = Dense(num_classes, activation='softmax')(dropout2)
model = Model(inputs=base_model.inputs, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy', 'top_k_categorical_accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
]



In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    steps_per_epoch=len(train_generator),
    callbacks=callbacks,
    validation_data=val_generator,
    validation_steps=len(val_generator),
)
model.save("model.h5")

Epoch 1/20


2024-12-08 04:23:12.832907: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


249/249 [==============================] - ETA: 0s - loss: 4.6150 - accuracy: 0.0746 - top_k_categorical_accuracy: 0.2767

2024-12-08 04:36:08.617317: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


249/249 [==============================] - 947s 4s/step - loss: 4.6150 - accuracy: 0.0746 - top_k_categorical_accuracy: 0.2767 - val_loss: 3.9547 - val_accuracy: 0.2424 - val_top_k_categorical_accuracy: 0.6234 - lr: 1.0000e-04
Epoch 2/20
249/249 [==============================] - 996s 4s/step - loss: 3.8641 - accuracy: 0.1636 - top_k_categorical_accuracy: 0.5087 - val_loss: 3.3998 - val_accuracy: 0.3135 - val_top_k_categorical_accuracy: 0.6878 - lr: 1.0000e-04
Epoch 3/20
249/249 [==============================] - 994s 4s/step - loss: 3.5106 - accuracy: 0.2210 - top_k_categorical_accuracy: 0.6087 - val_loss: 3.1492 - val_accuracy: 0.3393 - val_top_k_categorical_accuracy: 0.7235 - lr: 1.0000e-04
Epoch 4/20
249/249 [==============================] - 1009s 4s/step - loss: 3.3003 - accuracy: 0.2639 - top_k_categorical_accuracy: 0.6527 - val_loss: 3.0102 - val_accuracy: 0.3514 - val_top_k_categorical_accuracy: 0.7397 - lr: 1.0000e-04
Epoch 5/20
249/249 [==============================] - 1004

In [26]:
# save the model to a file
model.save("./model-data/model.h5")

In [27]:
# load the model from a file
from keras.models import load_model
model = load_model("./model-data/model.h5")

In [28]:
# test the model
testgen = ImageDataGenerator(rescale=1./255)
test_generator = testgen.flow_from_dataframe(
    dataframe=test_df,
    directory="./dataset",
    x_col="img_path",
    y_col="category",
    target_size=(256, 256),
    batch_size=128,
    class_mode="categorical",
    workers=4
)

Found 10904 validated image filenames belonging to 46 classes.


In [29]:
# test the model on the first batch of test data
model.evaluate(test_generator)

2024-12-10 01:02:00.591244: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


86/86 [==============================] - 511s 6s/step - loss: 2.4536 - accuracy: 0.4062 - top_k_categorical_accuracy: 0.8110


[2.4535539150238037, 0.4061812162399292, 0.8109868168830872]